In [1]:
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from pandas import DataFrame
from pandas.core.dtypes import common as com
from core.loader import Loader

from models.model_wrapper import ModelWrapper

from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO: Zkusit 3 stupně na NN, SVM a porovnat s LightGBM + XGBoost

# 4 x Malware
# 4x  Phishing
# + DGA (jeden stupen)

2025-05-01 21:22:33.803119: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-01 21:22:33.803159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-01 21:22:33.804443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-01 21:22:33.812241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-01 21:22:34.769452: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-05-01 21:22:37.566703: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-01 21:22:37.604391: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-01 21:22:37.607308: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
import numpy as np
import joblib
import shap
import matplotlib.pyplot as plt
from models.model_wrapper import ModelWrapper
from core.meta_nn import MetaNeuralClassifier
from core.validator import load_saved_split, load_train_split

ARCHITECTURES = ["XgBoost", "Lgbm", "feedforward"]
VERSION = "v1.1"
MALICIOUS_LABEL = "phishing"
STAGE = 3
VERIFICATION = True
META_MODEL_PATH = "./models/meta_nn_model"

# === Load data ===
x_train, x_test, y_train, y_test = load_train_split(STAGE, MALICIOUS_LABEL)
x_train_additional, y_test_additional = load_saved_split(STAGE, MALICIOUS_LABEL, folder="./data/", verification=VERIFICATION)

x_train = x_train[:int(len(x_train) * 0.10)]
y_train = y_train[:int(len(y_train) * 0.10)]
x_train = np.vstack((x_train, x_train_additional))
y_train = np.hstack((y_train, y_test_additional))
x_test = np.vstack((x_test, x_train_additional))
y_test = np.hstack((y_test, y_test_additional))

# === Model SHAP analysis ===
model_wrapper = ModelWrapper(model_dir="models")
shap_results = {}
combined_shap_values = []
combined_feature_names = []

x_sample_orig = x_test[:100]  # Raw input sample before any transformation

for arch in ARCHITECTURES:
    print(f"\n--- SHAP for {arch} ---")
    model = model_wrapper.load(
        arch_name=arch,
        label=MALICIOUS_LABEL,
        prefix=f"stage_{STAGE}",
        version=VERSION
    )

    x_sample = x_sample_orig.copy()
    if arch == "feedforward":
        scaler = joblib.load(f"scalers/{MALICIOUS_LABEL}_scaler.joblib")
        x_sample = scaler.transform(x_sample)
        explainer = shap.KernelExplainer(model.predict, x_sample)
    else:
        explainer = shap.TreeExplainer(model)

    shap_values = explainer.shap_values(x_sample)
    if isinstance(shap_values, list):  # For binary classification
        shap_values = shap_values[1]

    shap_results[arch] = {
        "values": shap_values,
        "data": x_sample,
        "explainer": explainer,
    }

    # Save individual plot
    shap.summary_plot(shap_values, x_sample, show=False)
    plt.title(f"SHAP Summary - {arch}")
    plt.savefig(f"shap_summary_{arch}.png", dpi=300, bbox_inches="tight")
    plt.clf()

    # Accumulate for combined analysis
    for i in range(shap_values.shape[1]):
        combined_feature_names.append(f"{arch}_f{i}")
    combined_shap_values.append(shap_values)

# === Combined SHAP summary plot for base models ===
print("\n--- Combined SHAP Summary for Base Models ---")
combined_shap_values = np.hstack(combined_shap_values)
shap.summary_plot(combined_shap_values, x_sample_orig[:100], feature_names=combined_feature_names, show=False)
plt.title("Combined SHAP Summary - All Base Models")
plt.savefig("shap_summary_combined_base.png", dpi=300, bbox_inches="tight")
plt.clf()

# === SHAP for Meta Neural Network ===
print("\n--- SHAP for Meta Neural Network ---")
meta_input = []
for arch in ARCHITECTURES:
    preds = shap_results[arch]["explainer"].model.predict(shap_results[arch]["data"])
    if arch == "feedforward":
        preds = (preds >= 0.5).astype(int).flatten()
    meta_input.append(preds)
meta_input = np.vstack(meta_input).T

meta_nn = MetaNeuralClassifier()
meta_nn.load(META_MODEL_PATH)

def predict_fn(x):
    return meta_nn.model.predict(x)

explainer = shap.KernelExplainer(predict_fn, meta_input[:100])
shap_values = explainer.shap_values(meta_input[:100])
shap_results["meta_nn"] = {
    "values": shap_values,
    "data": meta_input[:100],
    "explainer": explainer,
}

shap.summary_plot(shap_values, meta_input[:100], feature_names=ARCHITECTURES, show=False)
plt.title("SHAP Summary - Meta Neural Network")
plt.savefig("shap_summary_meta_nn.png", dpi=300, bbox_inches="tight")
plt.clf()
